In [ ]:
import os
import zipfile as zip
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from fuzzywuzzy import fuzz
import ast
import datetime
import dateparser as dp
from sklearn.metrics.pairwise import cosine_similarity
from concentrationMetrics import Index

In [ ]:
os.chdir("E:\data")

In [ ]:
#gcpd data
gcpd = pd.read_csv("GCPD_granular_data.txt")

In [ ]:
gcpd

In [ ]:
# top n pharmaceutical companies
top10 = ['novartis', 'pfizer', 'roche', 'sanofi', 'merck', 'johnson & johnson', 'glaxosmithkline', 'astrazeneca', 'gilead sciences', 'abbvie']
top25 = ['novartis', 'pfizer', 'roche', 'sanofi', 'merck', 'johnson & johnson', 'glaxosmithkline', 'astrazeneca', 'gilead sciences', 'abbvie', 'amgen', 'teva', 'bayer', 'eli lilly', 'novo nordisk', 'boehringer ingelheim', 'takeda', 'bristol-myers squibb', 'actavis', 'astellas pharma', 'baxter international', 'biogen idec', 'merck kgaa', 'mylan', 'daiichi sankyo']

In [ ]:
for i in top25:
    print(gcpd[gcpd['conml'].str.contains(i, case=False)]['conml'].unique())

In [ ]:
top10 = ['Novartis AG', 'Pfizer Inc', 'Roche Holding AG', 'Sanofi', 'Merck & Co Inc.', 'Johnson & Johnson', 'Glaxosmithkline PLC', 'Astrazeneca PLC' ,'Gilead Sciences Inc', 'AbbVie Inc']
# top25 = ['Novartis AG', 'Pfizer Inc', 'Roche Holding AG', 'Sanofi', 'Merck & Co Inc.', 'Johnson & Johnson', 'Glaxosmithkline PLC', 'Astrazeneca PLC' ,'Gilead Sciences Inc', 'AbbVie Inc', 'Amgen Inc', 'Teva Pharmaceutical Industries Ltd', 'Bayer AG', 'Eli Lilly and Co', 'Novo Nordisk A/S'
# []
# ['Takeda Pharmaceutical Co']
# ['Bristol-Myers Squibb Co']
# []
# ['Astellas Pharma Inc']
# ['Baxter International Inc']
# []
# ['Merck Kgaa']
# ['Mylan NV']
# ['Daiichi Sankyo Company Ltd']

In [ ]:
for i in top10:
    print(i)
    print(gcpd[gcpd['conml'].str.contains(i)]['assg_name'].unique())

In [ ]:
top10pat = gcpd[gcpd['conml'].isin(top10)]

In [ ]:
top10pat['date_issue'] = top10pat['date_issue'].apply(dp.parse)
top10pat['date_filing'] = top10pat['date_filing'].apply(dp.parse)

In [ ]:
top10pat

In [ ]:
top10pat.to_csv('top10pat.csv')

In [ ]:
top10pat = pd.read_csv('top10pat.csv', index_col=0)

In [ ]:
top10pat['date_issue'] = pd.to_datetime(top10pat['date_issue'])
top10pat['date_filing'] = pd.to_datetime(top10pat['date_filing'])

In [ ]:
#2010 to 2015
sampleframe = top10pat[(top10pat['date_issue']>=datetime.datetime(2010,1,1)) & (top10pat['date_issue']<datetime.datetime(2016,1,1))]

In [ ]:
sampleframe

In [ ]:
sampleframe = sampleframe.rename(columns={'nr_pt':'patent_id'})

In [ ]:
sampleframe['patent_id'] = sampleframe['patent_id'].astype(str)

In [ ]:
pi = pd.read_csv("patent_inventor.tsv", delimiter="\t")

In [ ]:
pi

In [ ]:
sampleframe2 = sampleframe.merge(pi, how='left', on='patent_id')

In [ ]:
sampleframe2

In [ ]:
sampleframe2.to_csv("sampleframe.csv")

In [ ]:
sampleframe2 = pd.read_csv('sampleframe.csv',index_col=0)

In [ ]:
ui = pd.DataFrame(sampleframe2['inventor_id'].unique(), columns=['inventor_id'])

In [ ]:
ui

In [ ]:
ui = ui.merge(pi[['patent_id','inventor_id']], how='left', on='inventor_id')

In [ ]:
ui

In [ ]:
ui[ui['inventor_id']=='7921949-4']

In [ ]:
ui.to_csv('top10inventors.csv')

In [ ]:
ipc = pd.read_csv("ipc3.csv", usecols=["patent_id","class"])

In [ ]:
ipc = ipc.drop_duplicates()

In [ ]:
ipc[ipc['patent_id']=='6926012']

In [ ]:
ui = ui.merge(ipc, how='left', on='patent_id')

In [ ]:
ui = ui.drop_duplicates()

In [ ]:
pd.get_dummies(ui['class'])

In [ ]:
ui = pd.concat([ui,pd.get_dummies(ui['class'])], axis=1)

In [ ]:
ui = ui.dropna(subset=['class'])

In [ ]:
ui = ui.merge(ui.dropna(subset=['class']).drop_duplicates(subset=['patent_id','class']).groupby('patent_id')['class'].count(), how='left', on='patent_id')

In [ ]:
ui.to_csv('uniqueinventors.csv')

In [ ]:
ui = pd.read_csv('uniqueinventors.csv', index_col=0)

In [ ]:
ui

In [ ]:
for i in ui.columns:
    if i in ['inventor_id', 'patent_id', 'class_x', 'class_y']:
        continue
    ui[i] = ui[i]/ui['class_y']

In [ ]:
ui['A00N']/ui['class_y']

In [ ]:
vectors = ui[ui.columns.difference(['patent_id','class_x','class_y'])].groupby('inventor_id').sum()

In [ ]:
vectors.to_csv('vectors.csv')

In [ ]:
vectors = pd.read_csv('vectors.csv', index_col=0)

In [ ]:
vectors

In [ ]:
sampleframe2 = sampleframe2.merge(vectors,how='left',on='inventor_id')

In [ ]:
sampleframe2

In [ ]:
sampleframe2 = sampleframe2.dropna()

In [ ]:
sampleframe2.to_csv('sampleframe2.csv')

In [ ]:
sampleframe2[sampleframe2.columns.difference(['gvkey','conml','assg_name','date_issue','date_filing','inventor_id','location_id'])].set_index('patent_id')

In [ ]:
sampleframe2['patent_id'] = sampleframe2['patent_id'].astype(str)

In [ ]:
grouped = sampleframe2.drop(['gvkey','conml','assg_name','date_issue','date_filing','inventor_id','location_id'], axis=1).set_index('patent_id').groupby('patent_id')

In [ ]:
cosine_similarity(grouped.get_group('9225375').drop('patent_id', axis=1))

In [ ]:
cosim = grouped.apply(cosine_similarity)

In [ ]:
cosim

In [ ]:
vectors

In [ ]:
grouped.size()

In [ ]:
x = grouped.get_group('7641093').iloc[0].to_numpy()
y = grouped.get_group('7641093').iloc[1].to_numpy()

In [ ]:
np.linalg.norm(x)

In [ ]:
np.linalg.norm(y)

In [ ]:
np.inner(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))

In [ ]:
np.inner([1,2,0,0],[2,1,0,0])

In [ ]:
np.min(cosim['7641093'])

In [ ]:
cosim['7641093']

In [ ]:
def cosim:
    

In [ ]:
plt.scatter(portfolios['patent_id'], portfolios['class'])

In [ ]:
portfolios.corr(method='pearson')

In [ ]:
portfolios.to_csv('top10portfolios.csv')

In [ ]:
plt.hist(portfolios['class'], range=(0,30), bins=30)

In [ ]:
portfolios[portfolios['class']==1]

In [ ]:
ui

In [ ]:
plt.hist(ui.groupby('patent_id')['inventor_id'].nunique(), range=(0,20), bins=20)

In [ ]:
ui['class1'] = ui['class'].str[0:1]
ui['class2'] = ui['class'].str[0:3]

In [ ]:
ui

In [ ]:
portfolios1 = pd.concat([ui.groupby('inventor_id')['patent_id'].nunique(),ui.groupby('inventor_id')['class'].nunique(),ui.groupby('inventor_id')['class1'].nunique(),ui.groupby('inventor_id')['class2'].nunique()], axis=1)

In [ ]:
portfolios1

In [ ]:
plt.scatter(portfolios1['patent_id'], portfolios1['class'])

In [ ]:
plt.scatter(portfolios1['patent_id'], portfolios1['class2'])

In [ ]:
plt.scatter(portfolios1['patent_id'], portfolios1['class1'])